# 🎓 RIS PhD Ultimate Research Dashboard

## Comprehensive Research Platform for RIS Probe-Based ML

**Version:** 2.0.0 (Phase 2 - MATLAB Integration Active)
**Author:** Your Name
**Last Updated:** January 2025

---

### 🚀 Features

#### Phase 1 (Core)
- ✅ 5 Configuration Tabs (System, Physics, Model, Training, Evaluation, Visualization)
- ✅ 19+ Pre-defined Model Architectures + Custom
- ✅ 6 Probe Types (continuous, binary, 2bit, hadamard, sobol, halton)
- ✅ Experiment Stacking & Batch Execution
- ✅ Transfer Learning Support
- ✅ 25+ Plot Types with Interactive Visualization
- ✅ Multi-Model & Multi-Seed Comparison
- ✅ Config Save/Load (JSON)

#### Phase 2 (MATLAB Integration)
- ✅ Dual Backend System (Python / MATLAB)
- ✅ MATLAB Engine Integration
- ✅ MathWorks Verified Toolboxes (Communications, 5G)
- ✅ Multiple Channel Scenarios (Rayleigh, CDL-RIS, TDL, Rician)
- ✅ Automatic Fallback to Python

---

### 📚 Quick Start Guide

1. **Run Cell 2** - Setup and verify environment
2. **Run Cell 3** - Initialize dashboard
3. **Configure** - Use tabs to set parameters
4. **Add to Stack** - Build experiment queue
5. **Run Stack** - Execute all experiments
6. **Run Cell 4** - View results and analysis

---

## Cell 1: Setup & Installation Check

Verify environment setup and dependencies.

In [1]:
from networkx.algorithms.tournament import score_sequence
from scipy.cluster.hierarchy import weighted
from torch.fx.experimental.symbolic_shapes import lru_cache

from tests.diagnose_runner_integration import criterion
# ============================================================================
# CELL 2: ENVIRONMENT SETUP & VERIFICATION
# ============================================================================

%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path

# Ensure project root is in path
project_root = Path(os.getcwd()).parent if 'notebooks' in os.getcwd() else Path(os.getcwd())
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("="*70)
print("🔧 RIS RESEARCH PLATFORM - ENVIRONMENT CHECK")
print("="*70)
print()

# Check Python version
print("[1/6] Python Version:")
print(f"   ✓ Python {sys.version.split()[0]}")
print()

# Check core dependencies
print("[2/6] Core Dependencies:")
required_packages = [
    'numpy', 'torch', 'matplotlib', 'seaborn',
    'scipy', 'pandas', 'ipywidgets'
]
for pkg in required_packages:
    try:
        __import__(pkg)
        print(f"   ✓ {pkg}")
    except ImportError:
        print(f"   ✗ {pkg} - MISSING!")
print()

# Check project modules
print("[3/6] Project Modules:")
project_modules = ['config', 'data', 'models', 'training', 'evaluation', 'dashboard', 'physics']
for mod in project_modules:
    try:
        __import__(mod)
        print(f"   ✓ {mod}")
    except ImportError as e:
        print(f"   ✗ {mod} - ERROR: {e}")
print()

# Check MATLAB availability (Phase 2)
print("[4/6] MATLAB Backend (Phase 2):")
try:
    import matlab.engine
    print("   ✓ MATLAB Engine for Python - AVAILABLE")
    print("   ✓ Phase 2 features: ENABLED")
except ImportError:
    print("   ⚠ MATLAB Engine - NOT AVAILABLE")
    print("   ℹ Phase 2 will use Python fallback")
print()

# Check GPU availability
print("[5/6] GPU Acceleration:")
try:
    import torch
    if torch.cuda.is_available():
        print(f"   ✓ CUDA available: {torch.cuda.get_device_name(0)}")
    else:
        print("   ℹ CPU only (no GPU detected)")
except:
    print("   ℹ CPU only")
print()

# Verify dashboard components
print("[6/6] Dashboard Components:")
try:
    from dashboard import create_complete_interface
    from dashboard.callbacks import setup_all_callbacks, setup_experiment_handlers
    print("   ✓ Dashboard interface")
    print("   ✓ Callback system")
    print("   ✓ Experiment runner")
except ImportError as e:
    print(f"   ✗ Dashboard ERROR: {e}")
print()

print("="*70)
print("✅ ENVIRONMENT CHECK COMPLETE")
print("="*70)
print()
print("Ready to proceed! Run Cell 3 to launch dashboard.")
print()

EXPERIMENT RUNNER DIAGNOSTIC

[STEP 1] Creating Configuration
----------------------------------------------------------------------
✓ Config created
  Training samples: 100
  Model: Baseline_MLP
  Hidden layers: [256, 128]

[STEP 2] Generating Data (Your Pipeline)
----------------------------------------------------------------------
✓ Probe bank generated
✓ Datasets generated
  Train: 100 samples
  Val: 20 samples

  Train data stats:
    masked_powers mean: 0.013322
    masked_powers std: 0.051185
    Unique labels: 52

[STEP 3] Creating DataLoaders
----------------------------------------------------------------------
✓ Input tensors created
  Train inputs shape: torch.Size([100, 128])
  Expected: (100, 128)
  Train labels shape: torch.Size([100])

  Input stats:
    Mean: 0.069161
    Std: 0.243146
    Contains NaN: False
    Contains Inf: False
✓ DataLoaders created
  Train batches: 7
  Val batches: 2

[STEP 4] Creating Model
------------------------------------------------------

In [2]:
# ============================================================================
# CELL 3: DASHBOARD INITIALIZATION
# ============================================================================

from IPython.display import display, clear_output
from dashboard import create_complete_interface
from dashboard.callbacks import setup_all_callbacks, setup_experiment_handlers

print("="*70)
print("🎛️ INITIALIZING RIS RESEARCH DASHBOARD")
print("="*70)
print()

# Create dashboard interface
print("Creating dashboard interface...")
complete_ui, widget_dict = create_complete_interface()

# Setup callbacks
print("Connecting interactive callbacks...")
setup_all_callbacks(widget_dict)

# Setup experiment handlers
print("Connecting experiment handlers...")
setup_experiment_handlers(widget_dict)

print()
print("="*70)
print("✅ DASHBOARD READY")
print("="*70)
print()
print("📋 Quick Tips:")
print("   • Use tabs to configure parameters")
print("   • Add experiments to stack with custom names")
print("   • Transfer learning: Select source experiment")
print("   • Phase 2: Switch to MATLAB backend in Physics tab")
print()

# Display dashboard
display(complete_ui)

🎛️ INITIALIZING RIS RESEARCH DASHBOARD

Creating dashboard interface...
Connecting interactive callbacks...
Connecting experiment handlers...

✅ DASHBOARD READY

📋 Quick Tips:
   • Use tabs to configure parameters
   • Add experiments to stack with custom names
   • Transfer learning: Select source experiment
   • Phase 2: Switch to MATLAB backend in Physics tab



In [ ]:
# DIAGNOSTIC TEST - Run this in a notebook cell
import torch
import torch.nn as nn
import numpy as np

# Simulate your current setup
K = 64
batch_size = 32

# Model output (logits)
logits = torch.randn(batch_size, K)

# Labels (probe indices)
labels = torch.randint(0, K, (batch_size,))

print("="*70)
print("DIAGNOSTIC TEST: Loss Function Check")
print("="*70)

# Test 1: MSELoss (what you currently have)
mse_loss = nn.MSELoss()
try:
    loss_mse = mse_loss(logits, labels.float().unsqueeze(1))
    print(f"\n✗ MSELoss: {loss_mse.item():.4f}")
    print("  Problem: Comparing logits to label indices doesn't make sense!")
except:
    print("\n✗ MSELoss: FAILED (dimension mismatch)")

# Test 2: CrossEntropyLoss (what you should have)
ce_loss = nn.CrossEntropyLoss()
try:
    loss_ce = ce_loss(logits, labels)
    print(f"\n✓ CrossEntropyLoss: {loss_ce.item():.4f}")
    print("  This is correct for classification!")
except Exception as e:
    print(f"\n✗ CrossEntropyLoss: FAILED - {e}")

# Test 3: Check if model is actually learning
print("\n" + "="*70)
print("TRAINING SIMULATION")
print("="*70)

# Create simple model
simple_model = nn.Sequential(
    nn.Linear(2*K, 128),
    nn.ReLU(),
    nn.Linear(128, K)
)

optimizer = torch.optim.Adam(simple_model.parameters(), lr=0.001)

print("\nWith CrossEntropyLoss:")
for epoch in range(5):
    optimizer.zero_grad()
    outputs = simple_model(torch.randn(batch_size, 2*K))
    loss = ce_loss(outputs, labels)
    loss.backward()
    optimizer.step()

    # Check accuracy
    preds = torch.argmax(outputs, dim=1)
    acc = (preds == labels).float().mean().item()

    print(f"  Epoch {epoch+1}: Loss={loss.item():.4f}, Acc={acc:.4f}")

print("\n" + "="*70)

In [ ]:
# In a NEW notebook cell (if you can open one) or new notebook entirely:
import os
import signal

# Get current process ID
pid = os.getpid()
print(f"Current process: {pid}")

# Force kill
os.kill(pid, signal.SIGTERM)

In [ ]:
import os
%cd ..
def print_minimal_tree(startpath):
    # Redundant folders and session-specific results to skip
    exclude_dirs = {'.git', '.idea', '__pycache__', '.venv', 'venv', 'results'}
    # File types to hide
    exclude_exts = {'.png', '.pt', '.json', '.pkl', '.ipynb_checkpoints'}

    for root, dirs, files in os.walk(startpath):
        # Prune redundant directories so the script doesn't even look inside them
        dirs[:] = [d for d in dirs if d not in exclude_dirs and not d.startswith('.')]

        level = root.replace(startpath, '').count(os.sep)
        indent = '│   ' * level

        # Print only the folder name
        curr_folder = os.path.basename(root) or os.path.basename(os.getcwd())
        print(f"{indent}├── {curr_folder}/")

        # Filter and print main files
        sub_indent = '│   ' * (level + 1)
        main_files = [f for f in files if not f.startswith('.')
                      and not any(f.endswith(ext) for ext in exclude_exts)]

        for i, f in enumerate(main_files):
            connector = '└── ' if (i == len(main_files) - 1 and not dirs) else '├── '
            print(f"{sub_indent}{connector}{f}")

if __name__ == "__main__":
    print_minimal_tree('.')


In [ ]:
import torch.nn.functional as F
o = F.one_hot(torch.tensor(0), num_classes=3)
print(o)

scores =torch.tensor([-4.1,5.2,0.7])
# one_hot_target =  torch.tensor([1,0,0])
one_hot_target =  o

criterion = nn.CrossEntropyLoss()
print(criterion(scores.double(), one_hot_target.double()))

model = nn.Sequential(nn.Linear(16, 8),
                      nn.Linear(8, 4),
                      nn.Linear(4, 2))
prediction = model()

criterion = nn.CrossEntropyLoss()
loss = criterion(op, one_hot_target)
loss.backward()

lr = 0.001

weight = model[0].weight
weight_grad = model[0].weight.grad

weight = weight - lr * weight_grad

bias = model[0].bias
bias_grad = model[0].bias.grad
bias = bias - lr * bias_grad


import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr=0.001)
optimizer.step()


In [ ]:
# tests/verify_expected_performance.py
import sys
import os
os.getcwd()
sys.path.insert(0, '.')

import numpy as np
from data.probe_generators import get_probe_bank
from data.data_generation import generate_limited_probing_dataset

N, K, M = 32, 64, 8
probe_bank = get_probe_bank('continuous', N=N, K=K, seed=42)

system_config = {'N': N, 'K': K, 'M': M, 'P_tx': 1.0, 'sigma_h_sq': 1.0, 'sigma_g_sq': 1.0}

data = generate_limited_probing_dataset(
    probe_bank=probe_bank,
    n_samples=1000,
    M=M,
    system_config=system_config,
    normalize=True,
    normalization_method='mean_sample',
    seed=42
)

# Check: Is best probe in the M observed probes?
best_in_observed = 0
for i in range(1000):
    best_probe = data['labels'][i]
    observed_probes = data['observed_indices'][i]
    if best_probe in observed_probes:
        best_in_observed += 1

print(f"Best probe in M={M} observed probes: {best_in_observed}/1000 = {best_in_observed/10:.1f}%")
print(f"Expected if random: M/K = {M}/{K} = {100*M/K:.1f}%")
print(f"\nThis is the CEILING for model accuracy - can't predict what you haven't measured!")

In [ ]:
# fix_emojis.py
import re

def remove_emojis_from_file(filepath):
    """Remove emoji characters from a file."""
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()

    # Remove common emojis
    emojis = ['🔧', '📊', '🎯', '🧠', '📈', '✅', '❌', '📡', '⚠️', '🔨', '✓', '⏹️', 'ℹ️']

    for emoji in emojis:
        content = content.replace(emoji + ' ', '')
        content = content.replace(emoji, '')

    # Save with utf-8 encoding
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(content)

    print(f"Fixed {filepath}")

# Fix both files
remove_emojis_from_file('dashboard/experiment_runner.py')
remove_emojis_from_file('data/data_generation.py')

print("Done! Emojis removed from both files.")

In [ ]:
# tests/proper_evaluation.py
import sys
sys.path.insert(0, '.')

import torch
import numpy as np
from data.probe_generators import get_probe_bank
from data.data_generation import generate_limited_probing_dataset
from models.base_models import BaselineMLPPredictor
from torch.utils.data import DataLoader, TensorDataset

N, K, M = 32, 64, 8
probe_bank = get_probe_bank('continuous', N=N, K=K, seed=42)

system_config = {'N': N, 'K': K, 'M': M, 'P_tx': 1.0, 'sigma_h_sq': 1.0, 'sigma_g_sq': 1.0}

# Generate test data
test_data = generate_limited_probing_dataset(
    probe_bank=probe_bank,
    n_samples=1000,
    M=M,
    system_config=system_config,
    normalize=True,
    normalization_method='mean_sample',
    seed=999
)

# Create trained model
train_data = generate_limited_probing_dataset(
    probe_bank=probe_bank,
    n_samples=10000,
    M=M,
    system_config=system_config,
    normalize=True,
    normalization_method='mean_sample',
    seed=42
)

train_inputs = torch.cat([
    torch.FloatTensor(train_data['masked_powers']),
    torch.FloatTensor(train_data['masks'])
], dim=1)
train_labels = torch.LongTensor(train_data['labels'])
train_loader = DataLoader(TensorDataset(train_inputs, train_labels), batch_size=128, shuffle=True)

model = BaselineMLPPredictor(2*K, [256, 128], K, 0.1, True)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("Training model...")
for epoch in range(20):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        loss = criterion(model(inputs), labels)
        loss.backward()
        optimizer.step()

# Evaluate with PROPER metrics
model.eval()
test_inputs = torch.cat([
    torch.FloatTensor(test_data['masked_powers']),
    torch.FloatTensor(test_data['masks'])
], dim=1)

with torch.no_grad():
    outputs = model(test_inputs)
    predictions = outputs.numpy()

# Metric 1: Top-1 accuracy (what you've been seeing)
top1_correct = 0
for i in range(len(test_data['labels'])):
    pred = np.argmax(predictions[i])
    if pred == test_data['labels'][i]:
        top1_correct += 1

top1_acc = top1_correct / len(test_data['labels'])

# Metric 2: Best-of-Observed (much more meaningful!)
best_obs_correct = 0
for i in range(len(test_data['labels'])):
    observed = test_data['observed_indices'][i]
    # Only consider predictions for observed probes
    observed_scores = predictions[i][observed]
    best_observed_idx = observed[np.argmax(observed_scores)]

    # Compare to actual best among observed
    observed_powers = test_data['powers_full'][i][observed]
    true_best_observed = observed[np.argmax(observed_powers)]

    if best_observed_idx == true_best_observed:
        best_obs_correct += 1

best_obs_acc = best_obs_correct / len(test_data['labels'])

# Metric 3: Power ratio (eta)
etas = []
for i in range(len(test_data['labels'])):
    observed = test_data['observed_indices'][i]
    observed_scores = predictions[i][observed]
    selected_probe = observed[np.argmax(observed_scores)]

    selected_power = test_data['powers_full'][i][selected_probe]
    optimal_power = test_data['optimal_powers'][i]

    eta = selected_power / optimal_power
    etas.append(eta)

mean_eta = np.mean(etas)

# Baselines
random_in_observed = M / K  # 12.5%
random_any = 1 / K  # 1.56%

print("\n" + "="*70)
print("PROPER EVALUATION METRICS")
print("="*70)
print(f"\n1. Top-1 Accuracy (predicting global best from M observations):")
print(f"   Model: {top1_acc*100:.1f}%")
print(f"   Theoretical ceiling: {random_in_observed*100:.1f}%")
print(f"   Random guess: {random_any*100:.1f}%")
print(f"   Model efficiency: {top1_acc/random_in_observed*100:.1f}% of theoretical max")

print(f"\n2. Best-of-Observed Accuracy (picking best among M=8):")
print(f"   Model: {best_obs_acc*100:.1f}%")
print(f"   Random: {100/M:.1f}%")
print(f"   → This shows the model IS learning patterns!")

print(f"\n3. Power Ratio (η - fraction of optimal power):")
print(f"   Model: {mean_eta:.3f}")
print(f"   Oracle (perfect): 1.000")
print(f"   Random M probes: ~0.45")

print("\n" + "="*70)
print("CONCLUSION: Your model is working correctly!")
print("="*70)

In [ ]:
# tests/diagnose_power_mismatch.py
import sys
sys.path.insert(0, '.')

import numpy as np
from data.probe_generators import get_probe_bank
from data.data_generation import generate_limited_probing_dataset

N, K, M = 32, 64, 8
probe_bank = get_probe_bank('continuous', N=N, K=K, seed=42)
system_config = {'N': N, 'K': K, 'M': M, 'P_tx': 1.0, 'sigma_h_sq': 1.0, 'sigma_g_sq': 1.0}

# Generate data
data = generate_limited_probing_dataset(
    probe_bank=probe_bank,
    n_samples=1000,
    M=M,
    system_config=system_config,
    normalize=True,
    normalization_method='mean_sample',
    seed=42
)

print("="*70)
print("POWER DISTRIBUTION ANALYSIS")
print("="*70)

# Check the actual power values
powers_full = data['powers_full']
optimal_powers = data['optimal_powers']

print(f"\nFull probe powers statistics:")
print(f"  Mean: {np.mean(powers_full):.4f}")
print(f"  Std: {np.std(powers_full):.4f}")
print(f"  Min: {np.min(powers_full):.4f}")
print(f"  Max: {np.max(powers_full):.4f}")

print(f"\nOptimal powers statistics:")
print(f"  Mean: {np.mean(optimal_powers):.4f}")
print(f"  Std: {np.std(optimal_powers):.4f}")
print(f"  Min: {np.min(optimal_powers):.4f}")
print(f"  Max: {np.max(optimal_powers):.4f}")

# Check ratio of max probe power to optimal
ratios = []
for i in range(1000):
    max_probe_power = np.max(powers_full[i])
    optimal = optimal_powers[i]
    ratios.append(max_probe_power / optimal)

print(f"\nRatio of best probe power to optimal power:")
print(f"  Mean: {np.mean(ratios):.4f}")
print(f"  Expected: ~0.25-0.30 for random RIS phases")

if np.mean(ratios) < 0.15:
    print(f"  ❌ PROBLEM: Ratio is too low!")
    print(f"     This suggests probes are not achieving good power")
    print(f"     Possible causes:")
    print(f"     1. Probe bank phases are not optimized")
    print(f"     2. Channel generation issue")
elif np.mean(ratios) > 0.9:
    print(f"  ❌ PROBLEM: Ratio is too high!")
    print(f"     This suggests probes are too similar to optimal")
    print(f"     The task is too easy - no learning needed")
else:
    print(f"  ✓ Ratio looks reasonable")

# Check if best observed probe achieves reasonable power
best_obs_ratios = []
for i in range(1000):
    observed = data['observed_indices'][i]
    observed_powers = powers_full[i][observed]
    best_observed_power = np.max(observed_powers)
    optimal = optimal_powers[i]
    best_obs_ratios.append(best_observed_power / optimal)

print(f"\nRatio of best OBSERVED probe to optimal:")
print(f"  Mean: {np.mean(best_obs_ratios):.4f}")
print(f"  This is what the model should achieve at minimum")

print("="*70)

In [ ]:
# tests/test_with_oracle_selection.py
import sys
sys.path.insert(0, '.')

import torch
import numpy as np
from data.probe_generators import get_probe_bank
from data.data_generation import generate_limited_probing_dataset
from models.base_models import BaselineMLPPredictor
from torch.utils.data import DataLoader, TensorDataset

N, K, M = 32, 64, 8
probe_bank = get_probe_bank('continuous', N=N, K=K, seed=42)
system_config = {'N': N, 'K': K, 'M': M, 'P_tx': 1.0, 'sigma_h_sq': 1.0, 'sigma_g_sq': 1.0}

print("="*70)
print("COMPARISON: Random vs Oracle Probe Selection")
print("="*70)

for selection_method in ['random', 'oracle_topM']:
    print(f"\n{'='*70}")
    print(f"Testing with: {selection_method}")
    print(f"{'='*70}")

    # Generate training data
    train_data = generate_limited_probing_dataset(
        probe_bank=probe_bank,
        n_samples=10000,
        M=M,
        system_config=system_config,
        normalize=True,
        normalization_method='mean_sample',
        selection_method=selection_method,
        seed=42
    )

    # Generate test data
    test_data = generate_limited_probing_dataset(
        probe_bank=probe_bank,
        n_samples=1000,
        M=M,
        system_config=system_config,
        normalize=True,
        normalization_method='mean_sample',
        selection_method=selection_method,
        seed=999
    )

    # Quick check of data quality
    best_obs_ratios = []
    for i in range(1000):
        observed = test_data['observed_indices'][i]
        observed_powers = test_data['powers_full'][i][observed]
        best_observed_power = np.max(observed_powers)
        optimal = test_data['optimal_powers'][i]
        best_obs_ratios.append(best_observed_power / optimal)

    print(f"\nData quality:")
    print(f"  Best observed / optimal: {np.mean(best_obs_ratios):.4f}")

    # Train model
    train_inputs = torch.cat([
        torch.FloatTensor(train_data['masked_powers']),
        torch.FloatTensor(train_data['masks'])
    ], dim=1)
    train_labels = torch.LongTensor(train_data['labels'])
    train_loader = DataLoader(TensorDataset(train_inputs, train_labels), batch_size=128, shuffle=True)

    model = BaselineMLPPredictor(2*K, [256, 128], K, 0.1, True)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(20):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            loss = criterion(model(inputs), labels)
            loss.backward()
            optimizer.step()

    # Evaluate
    model.eval()
    test_inputs = torch.cat([
        torch.FloatTensor(test_data['masked_powers']),
        torch.FloatTensor(test_data['masks'])
    ], dim=1)

    with torch.no_grad():
        outputs = model(test_inputs)
        predictions = outputs.numpy()

    # Calculate eta
    etas = []
    for i in range(len(test_data['labels'])):
        observed = test_data['observed_indices'][i]
        observed_scores = predictions[i][observed]
        selected_probe = observed[np.argmax(observed_scores)]

        selected_power = test_data['powers_full'][i][selected_probe]
        optimal_power = test_data['optimal_powers'][i]

        eta = selected_power / optimal_power
        etas.append(eta)

    mean_eta = np.mean(etas)

    print(f"\nModel performance:")
    print(f"  η (power ratio): {mean_eta:.4f}")
    print(f"  Improvement over random: {mean_eta/0.108:.2f}x")

print("\n" + "="*70)
print("CONCLUSION")
print("="*70)
print("If η is much higher with oracle_topM, your model works correctly!")
print("The issue is just that random probe selection gives poor starting points.")